# Feature Extraction for EMG signals

## using LSTM

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from models import LSTM_Emb_Classifier, EMG_Feature_Extractor
from utils.logger import logger
import tqdm

### Training

In [ ]:
BATCH_SIZE = 32
LR = 0.0001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
STEP_SIZE = 10
GAMMA = 0.1
NUM_EPOCHS = 50

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = 'mps'
    logger.info("------ USING APPLE SILICON GPU ------")
 

In [ ]:
# Parametri del modello
input_dim = 8
hidden_dim = 64
embedding_dim = 32
output_dim = 8  # Definisci il numero di classi

# Inizializzazione del modello, della loss function e dell'ottimizzatore
model = LSTM_Emb_Classifier(input_dim, hidden_dim, embedding_dim, output_dim)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

train_dataset = "" # Inserisci il path al dataset di training
train_loader = "" # Inserisci il dataloader per il training

val_dataset = ""
val_loader = ""

logger.info(f"Model: {model}")
logger.info(f"len train_dataset: {len(train_dataset)}")
logger.info(f"len train_loader: {len(train_loader)}")


In [ ]:
def evaluate(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            outputs, embeddings = model(x)
            _, predicted = torch.max(outputs, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
    accuracy = correct / total
    return accuracy


In [ ]:
model.train()
for epoch in range(NUM_EPOCHS):
        model.train()
        epoch_loss = [0.0, 0]
        for i_val,(x, y) in tqdm(enumerate(train_loader)):
            x, y = x.to(DEVICE), y.to(DEVICE)
            #logger.info(f"X: {x[0][0]}")
            # Category Loss
            #logger.info(f"X: {x.size()}")

            outputs, embeddings = model(x)
            # Log details about the outputs
            #logger.info(f"Output type: {cls_o.logits.shape}")
                
            criterion = nn.CrossEntropyLoss()
            loss = criterion(outputs, y.long())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss[0] += loss.item()
            epoch_loss[1] += x.size(0)
            if (i_val + 1) % (len(train_loader) // 5) == 0:
                logger.info("[{}/{}]".format(i_val + 1, len(train_loader)))
            
        scheduler.step()
        logger.info(f'[EPOCH {epoch+1}] Avg. Loss: {epoch_loss[0] / epoch_loss[1]}')


        #save checkpoint in a file
        if (epoch+1) % 10 == 0:
            train_accuracy = evaluate(model, train_loader, DEVICE)
            val_accuracy = evaluate(model, val_loader, DEVICE)
            logger.info(f'[EPOCH {epoch+1}] Train Accuracy: {train_accuracy}')
            logger.info(f'[EPOCH {epoch+1}] Val Accuracy: {val_accuracy}')
            torch.save(model.state_dict(), f'./saved_models/LSTM_Emb_Classifier/final_LSTM_Emb_epoch_{epoch+1}.pth')
        if (epoch+1) % STEP_SIZE == 0:
            logger.info(f'Current LR: {scheduler.get_last_lr()}')

### Extraction

In [ ]:
# ----------TO ADJUST-------------
extraction_dataset = ""
extraction_loader = ""


# Estrazione degli embeddings per i dati completi
model.load_state_dict(torch.load(f'./saved_models/LSTM_Emb_Classifier/final_LSTM_Emb_epoch_50.pth'))
with torch.no_grad():
    all_embeddings = []
    for i_val,(x, y) in tqdm(enumerate(extraction_loader)):
        _, embeddings = model(x)
        all_embeddings.append(embeddings)
    all_embeddings = torch.cat(all_embeddings)

print(all_embeddings.shape)  # Controlla la forma degli embeddings

## using Not Neural Network Feature Extraction